In [2]:
!pip install xgboost
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import ( accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, classification_report )

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 6.0 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
df_train = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

In [5]:
columns_to_drop = ['com_reg_ver_win_rate', 'customer_idx', 'historical_existing_cnt', 'id_strategic_ver', 'it_strategic_ver', 'idit_strategic_ver', 'product_subcategory', 'product_modelname', 'ver_win_rate_x', 'ver_win_ratio_per_bu']

df_train.drop(columns=columns_to_drop, inplace=True)
df_test.drop(columns=columns_to_drop, inplace=True)

In [7]:
def label_encoding(series: pd.Series) -> pd.Series:
    my_dict = {}

    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

In [8]:
label_columns = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "customer_country.1",
    "customer_position",
    "response_corporate",
    "expected_timeline",
]

In [9]:
df_all = pd.concat([df_train[label_columns], df_test[label_columns]])

for col in label_columns:
    df_all[col] = label_encoding(df_all[col])

for col in label_columns:
    df_train[col] = df_all.iloc[:len(df_train)][col]
    df_test[col] = df_all.iloc[len(df_train):][col]

In [10]:
x_train, x_val, y_train, y_val = train_test_split(
    df_train.drop("is_converted", axis=1),
    df_train["is_converted"],
    test_size=0.2,
    shuffle=True,
    random_state=400,
)

In [11]:
x_train['expected_timeline'] = x_train['expected_timeline'].fillna('nan').apply(lambda x: str(x).split()[0].lower())

x_train['customer_country'].fillna(pd.Series(np.random.choice(['United States', 'India'], p=[0.5, 0.5], size=len(x_train))), inplace=True)
x_train['customer_type'].fillna(pd.Series(np.random.choice(['Specifier/Influencer', 'Unknown'], p=[0.3, 0.7], size=len(x_train))), inplace=True)
x_train['customer_job'].fillna('Unknown', inplace=True)
x_train['inquiry_type'].fillna('request for partnership', inplace=True)
x_train['product_category'].fillna(pd.Series(np.random.choice(['interactive digital board', 'Unknown'], p=[0.5, 0.5], size=len(x_train))), inplace=True)

numeric_features = x_train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = x_train.select_dtypes(include=['object']).columns

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])


In [12]:
tree_model = DecisionTreeClassifier(random_state=400)
xgb_model = XGBClassifier(random_state=400)
gb_model = GradientBoostingClassifier(random_state=400)

In [28]:
model = VotingClassifier(estimators=[
    ('decision_tree', tree_model),
    ('xgboost', xgb_model),
    ('gradient_boosting', gb_model)
], voting='soft') # or hard

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                   ('classifier', model)])

pipeline.fit(x_train, y_train)

y_val_pred = pipeline.predict(x_val)

In [29]:
print("Classification Report:\n", classification_report(y_val, y_val_pred))
print("오차행렬:\n", confusion_matrix(y_val, y_val_pred, labels=[True, False]))
print("\n정확도: {:.4f}".format(accuracy_score(y_val, y_val_pred)))
print("정밀도: {:.4f}".format(precision_score(y_val, y_val_pred, labels=[True, False])))
print("재현율: {:.4f}".format(recall_score(y_val, y_val_pred)))
print("F1: {:.4f}".format(f1_score(y_val, y_val_pred, labels=[True, False])))

Classification Report:
               precision    recall  f1-score   support

       False       0.95      0.99      0.97     10913
        True       0.74      0.46      0.57       947

    accuracy                           0.94     11860
   macro avg       0.85      0.72      0.77     11860
weighted avg       0.94      0.94      0.94     11860

오차행렬:
 [[  438   509]
 [  156 10757]]

정확도: 0.9439
정밀도: 0.7374
재현율: 0.4625
F1: 0.5685


In [30]:
x_test = df_test.drop(["is_converted", "id"], axis=1)

In [31]:
x_test['expected_timeline'] = x_test['expected_timeline'].fillna('nan').apply(lambda x: str(x).split()[0].lower())

x_test['customer_country'].fillna(pd.Series(np.random.choice(['United States', 'India'], p=[0.5, 0.5], size=len(x_test))), inplace=True)
x_test['customer_type'].fillna(pd.Series(np.random.choice(['Specifier/Influencer', 'Unknown'], p=[0.3, 0.7], size=len(x_test))), inplace=True)
x_test['customer_job'].fillna('Unknown', inplace=True)
x_test['inquiry_type'].fillna('request for partnership', inplace=True)
x_test['product_category'].fillna(pd.Series(np.random.choice(['interactive digital board', 'Unknown'], p=[0.5, 0.5], size=len(x_test))), inplace=True)

In [32]:
test_pred = pipeline.predict(x_test)
sum(test_pred) # True로 예측된 개수

465

In [33]:
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)
